imports

In [1]:
from tqdm.notebook import tqdm as tqdm
import os
import json
import pandas as pd

from HGNN.train.configParser import ConfigParser, getModelName
from myhelpers import TrialStatistics

parameters

In [2]:
experimentsPath="/raid/elhamod/Fish/official_experiments/"
dataPath="/raid/elhamod/Fish/"
experimentName="Hard_notpretrained"
# experimentName="Hard_noaggregation"
# experimentName="Hard"
# experimentName="Easy"

cuda=0

file and changes

In [3]:
old_params_file = os.path.join(experimentsPath, experimentName, "params.json")

# Keep same order as ConfigParserWriter!
changes = {
#     'pretrained': False
}
removes = ['phylogeny_loss_epsilon', "noSpeciesBackprop", "phylogeny_loss", "unsupervisedOnTest"]

prepare

In [4]:
config_parser = ConfigParser(experimentsPath, dataPath, experimentName)
paramsIterator = config_parser.getExperiments()  
number_of_experiments = sum(1 for e in paramsIterator)

# populate a hash table of all experiments, indexed by trial hash
experiments = {}
for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
    num_of_models = experiment_params["numOfTrials"]
    for k in range(num_of_models):
#         print(experiment_params)
        old_trialHash = TrialStatistics.getTrialName(experiment_params, k)
        experiment_params_new = {**experiment_params, **changes} 
        for key in removes:
            experiment_params_new.pop(key, None)
#         print(experiment_params_new)
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)
        new_trialHash = TrialStatistics.getTrialName(experiment_params_new, k)
        experiments[old_trialHash] = {"trialHash": new_trialHash}
        
experiments

{'561cffae5e8eb405bfdf19f687fbb27008ffae6c0c9c35103b614cdf': {'trialHash': '5a1c1cb9ada98bf1bc733868da195dc1d1338c44e0098667996c129d'},
 '9260da9f7780479ba1396b6acf6ed5e0c866640a5f1b94e15c617107': {'trialHash': '07cf8b8f49af586a8e74be62c757e641592d78f1700a989e92f2eb3f'},
 'edff1486b1c4597c6ad247475e489c91ed39e59c3502b46755cdb8e2': {'trialHash': 'a0a77de87f71334d10585afcae0401d22211391bb522cb4255803d6a'},
 '213cd4ec3515779c1ce9790eccf5cbaa36862b433eddfe472a65b71a': {'trialHash': '1a4e8720316c2f253c437dad3bbb716bdb18f9dd52a008d351e71caf'},
 '8d62203d0bf3da07aa24d919ec29b6425118f1a3e83dce79dbeee896': {'trialHash': '1a2f280a22dad1045212ea3f8ad2e2363e16369697f57dc725b6ccc1'}}

Fix models

In [5]:
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        experiment_params_new = {**experiment_params, **changes} 
        for key in removes:
            experiment_params_new.pop(key, None)
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)
        print(experiment_params_new)

        num_of_models = experiment_params["numOfTrials"]
        with tqdm(total=num_of_models, desc="trial") as bar2:
            for k in range(num_of_models):
            
                # Fix model names
                old_model_name = getModelName(experiment_params, k)
                old_modelPath = os.path.join(experimentsPath, experimentName, old_model_name)
                new_model_name = getModelName(experiment_params_new, k)
                new_modelPath = os.path.join(experimentsPath, experimentName, new_model_name)

                try:
                    print(old_modelPath)
                    if os.path.exists(old_modelPath):
                        os.rename(old_modelPath, new_modelPath)
                        print("model", old_modelPath, '->', new_modelPath, "updated")

                        j = json.dumps(experiment_params_new)
                        json_fileName = os.path.join(new_modelPath, 'params.json')
                        f = open(json_fileName,"w")        
                        f.write(j)
                        f.close()  
                        print("json", json_fileName, "updated")
                    else:
                        raise
                    
                except:
                    print("model", old_model_name, "could not be fixed to", new_model_name)
                    pass  
                
                

                bar2.update()
                
                experiments[TrialStatistics.getTrialName(experiment_params, k)]['modelName'] = new_model_name

        bar.update()

{'image_path': 'Hard/curated_30_50', 'img_res': 448, 'augmented': True, 'batchSize': 100, 'learning_rate': 0.0001, 'numOfTrials': 5, 'fc_layers': 1, 'modelType': 'HGNN', 'lambda': 0.01, 'tl_model': 'ResNet18', 'link_layer': 'avgpool', 'adaptive_smoothing': True, 'adaptive_lambda': 0.01, 'adaptive_alpha': 0.9, 'pretrained': False}


/raid/elhamod/Fish/official_experiments/Hard_notpretrained/models/561cffae5e8eb405bfdf19f687fbb27008ffae6c0c9c35103b614cdf
model /raid/elhamod/Fish/official_experiments/Hard_notpretrained/models/561cffae5e8eb405bfdf19f687fbb27008ffae6c0c9c35103b614cdf -> /raid/elhamod/Fish/official_experiments/Hard_notpretrained/models/5a1c1cb9ada98bf1bc733868da195dc1d1338c44e0098667996c129d updated
json /raid/elhamod/Fish/official_experiments/Hard_notpretrained/models/5a1c1cb9ada98bf1bc733868da195dc1d1338c44e0098667996c129d/params.json updated
/raid/elhamod/Fish/official_experiments/Hard_notpretrained/models/9260da9f7780479ba1396b6acf6ed5e0c866640a5f1b94e15c617107
model /raid/elhamod/Fish/official_experiments/Hard_notpretrained/models/9260da9f7780479ba1396b6acf6ed5e0c866640a5f1b94e15c617107 -> /raid/elhamod/Fish/official_experiments/Hard_notpretrained/models/07cf8b8f49af586a8e74be62c757e641592d78f1700a989e92f2eb3f updated
json /raid/elhamod/Fish/official_experiments/Hard_notpretrained/models/07cf8b8f4

Fix results directory

In [6]:
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        experiment_params_new = {**experiment_params, **changes} 
        for key in removes:
            experiment_params_new.pop(key, None)
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)

        old_experimentHash = TrialStatistics.getTrialName(experiment_params)
        old_experimentPath = os.path.join(experimentsPath, experimentName, "results", old_experimentHash)
        new_experimentHash = TrialStatistics.getTrialName(experiment_params_new)
        new_experimentPath = os.path.join(experimentsPath, experimentName, "results", new_experimentHash)
        
        try:
            if os.path.exists(old_experimentPath):

                os.rename(old_experimentPath, new_experimentPath)
                print("result", old_experimentPath, '->', new_experimentPath, "updated")

                j = json.dumps(experiment_params_new)
                json_fileName = os.path.join(new_experimentPath, 'params.json')
                f = open(json_fileName,"w")        
                f.write(j)
                f.close()  
                print("json", json_fileName, "updated")
            else:
                raise
                
        except:
            print("result", old_experimentHash, "could not be fixed to",new_experimentHash)
            pass 
        
        experiments[TrialStatistics.getTrialName(experiment_params, 0)]['experimentHash'] = new_experimentHash
        
        bar.update()

result /raid/elhamod/Fish/official_experiments/Hard_notpretrained/results/f60977adc490982166beb6abd461d6ae085432f9b146fab8b4b2a9e5 -> /raid/elhamod/Fish/official_experiments/Hard_notpretrained/results/331fdf718eccc581e40a50d4677aad181b371dd30263c35a143f74f0 updated
json /raid/elhamod/Fish/official_experiments/Hard_notpretrained/results/331fdf718eccc581e40a50d4677aad181b371dd30263c35a143f74f0/params.json updated



Fix experiments file

In [7]:
# open experiments.csv file
experimentsFileNameAndPath=os.path.join(experimentsPath, "experiments.csv")
experiments_file= pd.read_csv(experimentsFileNameAndPath)

# For all experiments
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        
        numOfTrials = experiment_params["numOfTrials"]
        trialHash_0 = TrialStatistics.getTrialName(experiment_params, 0) # used for keys who don't change for different trials
        experiment_params_new = {**experiment_params, **changes} 
        for key in removes:
            experiment_params_new.pop(key, None)
        print(experiment_params_new)
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)
        print(experiment_params_new)
        
        # For all trials
        with tqdm(total=num_of_models, desc="trial") as bar2:
            for j in range(numOfTrials):
                
                trialHash = TrialStatistics.getTrialName(experiment_params, j)
#                 try:
                    # If old entry doesn't exist, nothing to update
    #                     record_exists = not (experiments_file[experiments_file['trialHash'] == trialHash][experiments_file['experimentName'] == experimentName]).empty
    #                     if record_exists:

                # drop old row and caluclate new row
                new_trialHash = experiments[trialHash]["trialHash"]
                experiments_file.drop(experiments_file[experiments_file['trialHash'] == trialHash][experiments_file['experimentName'] == experimentName].index, inplace = True) 
#                     print(experiments)
                row_information = {
                    'experimentName': experimentName,
                    'modelName': experiments[trialHash]["modelName"],
#                     'datasetName': experiments[trialHash_0]["datasetName"],
                    'experimentHash': experiments[trialHash_0]["experimentHash"],
                    'trialHash': new_trialHash
                }
                row_information = {**row_information, **experiment_params_new} 

                # error handling to avoid duplicates
                record_exists = not (experiments_file[experiments_file['trialHash'] == new_trialHash][experiments_file['experimentName'] == experimentName]).empty
                if record_exists:
                    experiments_file.drop(experiments_file[experiments_file['trialHash'] == new_trialHash][experiments_file['experimentName'] == experimentName].index, inplace = True) 

                # update with new entry    
                experiments_file = experiments_file.append(pd.DataFrame(row_information, index=[0]), ignore_index = True)
                print("trialHash", trialHash, "->", new_trialHash, "updated")
#                 except:
#                     print("trialHash", trialHash, "could not be updated!")
#                     pass
                bar2.update()
        bar.update()

# resave file
experiments_file.to_csv(experimentsFileNameAndPath, header=True, index=False)

{'image_path': 'Hard', 'img_res': 448, 'augmented': True, 'batchSize': 100, 'learning_rate': 0.0001, 'numOfTrials': 5, 'fc_layers': 1, 'modelType': 'HGNN', 'lambda': 0.01, 'tl_model': 'ResNet18', 'link_layer': 'avgpool', 'adaptive_smoothing': True, 'adaptive_lambda': 0.01, 'adaptive_alpha': 0.9, 'pretrained': False}
{'image_path': 'Hard/curated_30_50', 'img_res': 448, 'augmented': True, 'batchSize': 100, 'learning_rate': 0.0001, 'numOfTrials': 5, 'fc_layers': 1, 'modelType': 'HGNN', 'lambda': 0.01, 'tl_model': 'ResNet18', 'link_layer': 'avgpool', 'adaptive_smoothing': True, 'adaptive_lambda': 0.01, 'adaptive_alpha': 0.9, 'pretrained': False}


trialHash 561cffae5e8eb405bfdf19f687fbb27008ffae6c0c9c35103b614cdf -> 5a1c1cb9ada98bf1bc733868da195dc1d1338c44e0098667996c129d updated
trialHash 9260da9f7780479ba1396b6acf6ed5e0c866640a5f1b94e15c617107 -> 07cf8b8f49af586a8e74be62c757e641592d78f1700a989e92f2eb3f updated
trialHash edff1486b1c4597c6ad247475e489c91ed39e59c3502b46755cdb8e2 -> a0a77de87f71334d10585afcae0401d22211391bb522cb4255803d6a updated
trialHash 213cd4ec3515779c1ce9790eccf5cbaa36862b433eddfe472a65b71a -> 1a4e8720316c2f253c437dad3bbb716bdb18f9dd52a008d351e71caf updated
trialHash 8d62203d0bf3da07aa24d919ec29b6425118f1a3e83dce79dbeee896 -> 1a2f280a22dad1045212ea3f8ad2e2363e16369697f57dc725b6ccc1 updated




/home/elhamod/melhamodenv3/lib/python3.6/site-packages/ipykernel_launcher.py:30: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
/home/elhamod/melhamodenv3/lib/python3.6/site-packages/ipykernel_launcher.py:42: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


Fix params file

In [8]:
experiment_list = []
with tqdm(total=number_of_experiments, desc="experiment") as bar:
    for i, experiment_params in enumerate(config_parser.getExperiments(fixExperiments=False)):
        
        experiment_params_new = {**experiment_params, **changes} 
        for key in removes:
            experiment_params_new.pop(key, None)
        experiment_params_new = config_parser.fixExperimentParams(experiment_params_new)
        print(experiment_params_new)
        experiment_list.append(experiment_params_new)
        bar.update()


j = json.dumps({"experimentList": experiment_list})
f = open(old_params_file,"w")        
f.write(j)
f.close()  
print("json", old_params_file, "updated")

{'image_path': 'Hard/curated_30_50', 'img_res': 448, 'augmented': True, 'batchSize': 100, 'learning_rate': 0.0001, 'numOfTrials': 5, 'fc_layers': 1, 'modelType': 'HGNN', 'lambda': 0.01, 'tl_model': 'ResNet18', 'link_layer': 'avgpool', 'adaptive_smoothing': True, 'adaptive_lambda': 0.01, 'adaptive_alpha': 0.9, 'pretrained': False}

json /raid/elhamod/Fish/official_experiments/Hard_notpretrained/params.json updated
